In [8]:
import pandas as pd
import itertools
from support_functions import log_progress

# Predicting drug-target interactions
___
## Load the data from top-4 cell lines

In [9]:
all_pair_sig_ids = pd.DataFrame.from_csv('checkpoint_files/top_4_all_pair_sig_ids.csv')
cpd_lm_sigs = pd.DataFrame.from_csv('checkpoint_files/top_4_cpd_lm_sigs.csv')
target_kd_lm_sigs = pd.DataFrame.from_csv('checkpoint_files/top_4_target_kd_lm_sigs.csv')

In [10]:
# combine cpd and kd sigs into one dataframe
lm_sigs = pd.concat([cpd_lm_sigs,target_kd_lm_sigs], axis=1)

### Compute cpd-kd pearson correlations

In [11]:
# define the pairs we'll take the correlation between
cpd_col_names = sorted([ c for c in all_pair_sig_ids.columns if 'cpd_' in c ])
target_col_names = sorted([ c for c in all_pair_sig_ids.columns if 'target_' in c ])
cpd_target_cell_pairs = [ list(p) for p in zip(cpd_col_names,target_col_names) ]

In [12]:
# extract the cell lines
cell_lines = [ c.split('_')[1] for c in cpd_col_names ]

# initialize empty dataframe to store correlation values
training_data = all_pair_sig_ids[['pert_id', 'target', 'label']].reset_index(drop=True)
for cell in cell_lines: 
    training_data[cell] = pd.Series(0, index=training_data.index)

In [13]:
# extract the actual signatures and compute correlations
for index, row in log_progress(all_pair_sig_ids.iterrows(), every=1):
    for pair in cpd_target_cell_pairs:
        cell_line = pair[0].split('_')[1]
        [sig_id_1, sig_id_2] = row[pair].values
        corr = lm_sigs[sig_id_1].corr(lm_sigs[sig_id_2])
        training_data.loc[index, cell_line] = corr

In [16]:
training_data.to_csv('checkpoint_files/top_4_correlation_training_data.csv')

In [1]:
training_data.sample(n=10)

NameError: name 'training_data' is not defined